In [1]:
from faker import Faker
from faker import Faker
from PIL import Image, ImageDraw, ImageFont
import random
import os
import csv

fake = Faker('es_MX')

In [2]:
# Carpeta de destino para guardar las imágenes
CARPETA_DESTINO = r"C:\Users\Adrian\Desktop\data\INEs Dataset"

# Crear la carpeta si no existe
os.makedirs(CARPETA_DESTINO, exist_ok=True)

In [3]:
# Función para eliminar acentos
def eliminar_acentos(texto):
    reemplazos = {
        'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'Ü': 'U', 'ü': 'u', 'Ñ': 'N', 'ñ': 'n'
    }
    for acento, sin_acento in reemplazos.items():
        texto = texto.replace(acento, sin_acento)
    return texto

In [4]:
# Función para generar un CURP válido en formato
def generar_curp():
    letras = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    numeros = "0123456789"
    
    # Formato básico de CURP mexicano (simplificado)
    curp = ""
    curp += ''.join(random.choices(letras, k=4))  # Primeras 4 letras (apellidos y nombre)
    curp += ''.join(random.choices(numeros, k=6))  # Fecha (AAMMDD)
    curp += random.choice("HM")  # Sexo
    curp += ''.join(random.choices(letras, k=2))  # Estado
    curp += ''.join(random.choices(letras, k=3))  # Consonantes internas
    curp += random.choice(numeros + letras)  # Homoclave
    curp += ''.join(random.choices(numeros, k=1))  # Dígito verificador
    
    return curp

In [5]:
# Función para generar un número de elector válido
def generar_clave_elector():
    letras = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    numeros = "0123456789"
    
    # Formato típico de clave de elector
    clave = ""
    clave += ''.join(random.choices(letras, k=6))  # Primeras letras (apellidos y nombre)
    clave += ''.join(random.choices(numeros, k=8))  # Fecha y código
    clave += ''.join(random.choices(letras + numeros, k=4))  # Homoclave
    
    return clave

In [6]:
# Función para generar abreviación de estado
def abreviar_estado(estado):
    # Abreviaturas de estados mexicanos
    abreviaturas = {
        "AGUASCALIENTES": "AGS.",
        "BAJA CALIFORNIA": "BC.",
        "BAJA CALIFORNIA SUR": "BCS.",
        "CAMPECHE": "CAMP.",
        "CHIAPAS": "CHIS.",
        "CHIHUAHUA": "CHIH.",
        "COAHUILA": "COAH.",
        "COLIMA": "COL.",
        "DISTRITO FEDERAL": "CDMX.",
        "CIUDAD DE MEXICO": "CDMX.",
        "DURANGO": "DGO.",
        "GUANAJUATO": "GTO.",
        "GUERRERO": "GRO.",
        "HIDALGO": "HGO.",
        "JALISCO": "JAL.",
        "MEXICO": "MEX.",
        "ESTADO DE MEXICO": "MEX.",
        "MICHOACAN": "MICH.",
        "MORELOS": "MOR.",
        "NAYARIT": "NAY.",
        "NUEVO LEON": "NL.",
        "OAXACA": "OAX.",
        "PUEBLA": "PUE.",
        "QUERETARO": "QRO.",
        "QUINTANA ROO": "Q.ROO.",
        "SAN LUIS POTOSI": "SLP.",
        "SINALOA": "SIN.",
        "SONORA": "SON.",
        "TABASCO": "TAB.",
        "TAMAULIPAS": "TAMPS.",
        "TLAXCALA": "TLAX.",
        "VERACRUZ": "VER.",
        "YUCATAN": "YUC.",
        "ZACATECAS": "ZAC."
    }
    
    # Si el estado está en la lista, usar su abreviatura, de lo contrario abreviar las primeras 3 letras
    estado_limpio = eliminar_acentos(estado.upper())
    if estado_limpio in abreviaturas:
        return abreviaturas[estado_limpio]
    else:
        return estado_limpio[:3] + "."

In [7]:
def generar_direccion_completa():
    # Generar calle sin acentos
    calle = eliminar_acentos(fake.street_name().upper())
    
    # Generar número exterior con variedad de formatos
    formatos_ext = [
        str(random.randint(1, 9999)),  # Número simple: 123
        str(random.randint(1, 9999)) + "-" + str(random.randint(1, 99)),  # Con guión: 123-45
        str(random.randint(1, 9999)) + random.choice("ABCDEFGHIJ"),  # Con letra: 123A
        "S/N",  # Sin número
        "MZ " + str(random.randint(1, 99)) + " LT " + str(random.randint(1, 99)),  # Formato manzana-lote
        "KM " + str(random.randint(1, 100)) + "." + str(random.randint(0, 9))  # Kilómetro carretera
    ]
    num_ext = random.choice(formatos_ext)
    
    # Decidir si tiene número interior (30% de probabilidad)
    tiene_interior = random.random() < 0.3
    
    # Generar número interior con variedad
    num_int = ""
    if tiene_interior:
        formatos_int = [
            "INT " + str(random.randint(1, 999)),  # INT 123
            "DEPTO " + str(random.randint(1, 999)),  # DEPTO 123
            "DEPTO " + random.choice("ABCDEFGHIJ"),  # DEPTO A
            random.choice("ABCDEFGHIJ"),  # Sólo letra: A
            "PISO " + str(random.randint(1, 20)),  # PISO 3
            "EDIFICIO " + str(random.randint(1, 20)) + " DEPTO " + str(random.randint(1, 99))  # EDIFICIO 3 DEPTO 12
        ]
        num_int = random.choice(formatos_int)
    
    # Generar el resto de componentes
    colonia = eliminar_acentos(fake.city_suffix().upper() + " " + fake.city_prefix().upper())
    cp = str(random.randint(10000, 99999))  # Códigos postales en México son de 5 dígitos
    municipio = eliminar_acentos(fake.city().upper())
    estado = eliminar_acentos(fake.state().upper())
    estado_abreviado = abreviar_estado(estado)
    
    # Formatear la dirección completa
    direccion_linea1 = f"{calle} {num_ext}"
    if tiene_interior:
        direccion_linea1 += f", {num_int}"
        
    direccion_linea2 = f"{colonia} {cp}"
    direccion_linea3 = f"{municipio}, {estado_abreviado}"
    
    return {
        "direccion1": direccion_linea1,
        "direccion2": direccion_linea2,
        "direccion3": direccion_linea3
    }

In [8]:
# Función para generar datos de persona falsos
def generar_datos_persona():
    # Generar nombre y apellidos separados
    nombre = eliminar_acentos(fake.first_name().upper())
    apellido1 = eliminar_acentos(fake.last_name().upper())
    apellido2 = eliminar_acentos(fake.last_name().upper())
    
    # Determinar género y edad
    genero = random.choice(["H", "M"])
    fecha_nac = fake.date_of_birth(minimum_age=18, maximum_age=90)

    # Usar la nueva función para generar dirección
    direccion = generar_direccion_completa()
    
    seccion = str(random.randint(1, 9999)).zfill(4)
    
    # Vigencia como rango de años
    año_inicio = random.randint(2020, 2030)
    año_fin = año_inicio + 10
    vigencia = f"{año_inicio}-{año_fin}"
    
    # Año de registro con dos dígitos adicionales (mes)
    anio_registro = str(random.randint(2000, 2023))
    mes_registro = str(random.randint(1, 12)).zfill(2)  # Asegura que tenga 2 dígitos
    registro = f"{anio_registro} {mes_registro}"
    
    # Fecha de nacimiento en formato DD/MM/YYYY
    fecha_nacimiento = fecha_nac.strftime('%d/%m/%Y')
    
    return {
        "apellido1": apellido1,
        "apellido2": apellido2,
        "nombre": nombre,
        "clave_elector": generar_clave_elector().upper(),
        "curp": generar_curp().upper(),
        "fecha_nacimiento": fecha_nacimiento,
        "direccion1": direccion["direccion1"],
        "direccion2": direccion["direccion2"],
        "direccion3": direccion["direccion3"],
        "sexo": genero,
        "seccion": seccion,
        "vigencia": vigencia,
        "registro": registro
    }

In [9]:
# Cargar plantilla de INE y generar una falsa
def generar_ine_falsa(persona, output_filename):
    try:
        # Cargar imagen de plantilla
        imagen = Image.open("plantilla_ine.png")
        
        # Convertir a RGB si está en modo RGBA
        if imagen.mode == 'RGBA':
            imagen = imagen.convert('RGB')
            
        draw = ImageDraw.Draw(imagen)
        
        # Configurar fuentes
        try:
            # Todas en negrita si es posible
            fuente_etiqueta = ImageFont.truetype("arial.ttf", size=12)  # Para etiquetas
            fuente_dato = ImageFont.truetype("arial.ttf", size=14)     # Para los datos
            fuente_grande = ImageFont.truetype("arial.ttf", size=16)   # Para nombre
        except IOError:
            fuente_etiqueta = ImageFont.load_default()
            fuente_dato = ImageFont.load_default()
            fuente_grande = ImageFont.load_default()
        
        # Ajustar posiciones para dejar espacio para la foto (mover todo a la derecha)
        offset_x = 270  # Aumentado para dejar más espacio para la foto
        offset_y = 170
        
        # NOMBRE
        draw.text((offset_x, offset_y), "NOMBRE", font=fuente_etiqueta, fill="black")
        draw.text((offset_x, offset_y + 15), persona["apellido1"], font=fuente_grande, fill="black")
        draw.text((offset_x, offset_y + 32), persona["apellido2"], font=fuente_grande, fill="black")
        draw.text((offset_x, offset_y + 49), persona["nombre"], font=fuente_grande, fill="black")
        
        # SEXO - reposicionado para no encimarse
        draw.text((660, offset_y), "SEXO", font=fuente_etiqueta, fill="black")
        draw.text((696, offset_y - 1.5), persona["sexo"], font=fuente_dato, fill="black")
        
        # DOMICILIO - subido
        draw.text((offset_x, offset_y + 90), "DOMICILIO", font=fuente_etiqueta, fill="black")
        draw.text((offset_x, offset_y + 105), persona["direccion1"], font=fuente_grande, fill="black")
        draw.text((offset_x, offset_y + 122), persona["direccion2"], font=fuente_grande, fill="black")
        draw.text((offset_x, offset_y + 139), persona["direccion3"], font=fuente_grande, fill="black")
        
        # CLAVE DE ELECTOR (en la misma línea)
        draw.text((offset_x, offset_y + 180), "CLAVE DE ELECTOR", font=fuente_etiqueta, fill="black")
        draw.text((offset_x + 130, offset_y + 176), persona["clave_elector"], font=fuente_grande, fill="black")
        
        # AÑO DE REGISTRO (en la misma línea que CLAVE DE ELECTOR)
        draw.text((offset_x + 240, offset_y + 197), "AÑO DE REGISTRO", font=fuente_etiqueta, fill="black")
        draw.text((offset_x + 240, offset_y + 210), persona["registro"], font=fuente_grande, fill="black")
        
        # CURP (en la misma línea)
        draw.text((offset_x, offset_y + 197), "CURP", font=fuente_etiqueta, fill="black")
        draw.text((offset_x, offset_y + 210), persona["curp"], font=fuente_grande, fill="black")
        
        # FECHA DE NACIMIENTO (en la misma línea)
        draw.text((offset_x, offset_y + 229), "FECHA DE NACIMIENTO", font=fuente_etiqueta, fill="black")
        draw.text((offset_x, offset_y + 242), persona["fecha_nacimiento"], font=fuente_grande, fill="black")
        
        # SECCIÓN (en la misma línea)
        draw.text((offset_x + 160, offset_y + 229), "SECCIÓN", font=fuente_etiqueta, fill="black")
        draw.text((offset_x + 160, offset_y + 242), persona["seccion"], font=fuente_grande, fill="black")
        
        # VIGENCIA (en la misma línea)
        draw.text((offset_x + 240, offset_y + 229), "VIGENCIA", font=fuente_etiqueta, fill="black")
        draw.text((offset_x + 240, offset_y + 242), persona["vigencia"], font=fuente_grande, fill="black")
        
        # Ruta completa para guardar el archivo
        ruta_completa = os.path.join(CARPETA_DESTINO, output_filename)
        
        # Guardar la imagen
        imagen.save(ruta_completa)
        print(f"INE falsa generada: {ruta_completa}")
        return True
    except Exception as e:
        print(f"Error al generar INE falsa: {e}")
        return False

In [10]:
def exportar_datos_a_csv(personas, ruta_csv):
    """
    Exporta los datos de las personas a un archivo CSV
    
    Args:
        personas: Lista de diccionarios con los datos de las personas
        ruta_csv: Ruta donde se guardará el archivo CSV
    """
    # Definir los encabezados del CSV
    encabezados = [
        'Archivo_Imagen', 'Nombre', 'Segundo Nombre', 'Apellido Paterno', 'Apellido Materno',
        'Estado', 'Municipio', 'Colonia', 'Calle', 'Numero Ext.', 'Numero Int.',
        'Codigo Postal', 'CURP', 'Clave Elector', 'Fecha de Nacimiento',
        'Seccion', 'Vigencia Ano Inicio', 'Vigencia Ano Final', 'Ano Registro', 'Mes Registro'
    ]
    
    try:
        with open(ruta_csv, 'w', newline='', encoding='utf-8') as archivo_csv:
            writer = csv.DictWriter(archivo_csv, fieldnames=encabezados)
            writer.writeheader()
            
            for persona in personas:

                # Separar nombre en primer y segundo nombre (si existe)
                partes_nombre = persona['nombre'].split()
                primer_nombre = partes_nombre[0] if partes_nombre else ""
                segundo_nombre = " ".join(partes_nombre[1:]) if len(partes_nombre) > 1 else ""
                
                # Separar municipio y estado de la dirección3
                partes_dir3 = persona['direccion3'].split(',')
                municipio = partes_dir3[0].strip() if partes_dir3 else ""
                estado_abreviado = partes_dir3[1].strip() if len(partes_dir3) > 1 else ""
                # Quitar el punto de la abreviatura del estado
                if estado_abreviado.endswith('.'):
                    estado = estado_abreviado[:-1]  # Esta es la versión sin punto
                else:
                    estado = estado_abreviado
                
                # Separar colonia y CP de la dirección2
                partes_dir2 = persona['direccion2'].rsplit(' ', 1)
                colonia = partes_dir2[0].strip() if partes_dir2 else ""
                cp = partes_dir2[1].strip() if len(partes_dir2) > 1 else ""
                
                # Separar calle y número exterior/interior de la dirección1
                partes_dir1 = persona['direccion1'].split(', ')
                calle_num_ext = partes_dir1[0] if partes_dir1 else ""
                num_int = partes_dir1[1] if len(partes_dir1) > 1 else ""
                
                # Separar calle y número exterior
                partes_calle = calle_num_ext.rsplit(' ', 1)
                calle = partes_calle[0].strip() if len(partes_calle) > 1 else calle_num_ext
                num_ext = partes_calle[1].strip() if len(partes_calle) > 1 else ""
                
                # Separar año y mes de registro
                partes_registro = persona['registro'].split()
                anio_registro = partes_registro[0] if partes_registro else ""
                mes_registro = partes_registro[1] if len(partes_registro) > 1 else ""
                
                # Separar año inicio y fin de vigencia
                partes_vigencia = persona['vigencia'].split('-')
                vigencia_inicio = partes_vigencia[0] if partes_vigencia else ""
                vigencia_fin = partes_vigencia[1] if len(partes_vigencia) > 1 else ""
                
                # Crear fila para CSV
                fila = {
                    'Archivo_Imagen': persona.get('archivo_imagen', ''),
                    'Nombre': primer_nombre,
                    'Segundo Nombre': segundo_nombre,
                    'Apellido Paterno': persona['apellido1'],
                    'Apellido Materno': persona['apellido2'],
                    'Estado': estado,
                    'Municipio': municipio,
                    'Colonia': colonia,
                    'Calle': calle,
                    'Numero Ext.': num_ext,
                    'Numero Int.': num_int,
                    'Codigo Postal': cp,
                    'CURP': persona['curp'],
                    'Clave Elector': persona['clave_elector'],
                    'Fecha de Nacimiento': persona['fecha_nacimiento'],
                    'Seccion': persona['seccion'],
                    'Vigencia Ano Inicio': vigencia_inicio,
                    'Vigencia Ano Final': vigencia_fin,
                    'Ano Registro': anio_registro,
                    'Mes Registro': mes_registro
                }
                writer.writerow(fila)
                
        print(f"Archivo CSV generado exitosamente en: {ruta_csv}")
        return True
    except Exception as e:
        print(f"Error al generar el archivo CSV: {e}")
        return False

In [11]:
def generar_dataset(cantidad):
    generados = 0
    personas = []  # Lista para almacenar los datos de las personas
    
    for i in range(cantidad):
        # Generar nombre de archivo
        nombre_archivo = f"ine_falsa_{i+1:03d}.jpg"
        
        # Generar datos de persona y agregar el nombre de archivo
        persona = generar_datos_persona()
        persona['archivo_imagen'] = nombre_archivo  # Añadir el nombre de archivo a los datos
        
        personas.append(persona)  # Guardar los datos
        
        if generar_ine_falsa(persona, nombre_archivo):
            generados += 1
    
    print(f"Dataset generado: {generados} de {cantidad} INEs falsas")
    print(f"Las imágenes se guardaron en: {CARPETA_DESTINO}")
    
    # Exportar los datos a CSV en el directorio de trabajo actual
    ruta_csv = "datos_ines.csv"  # Se guardará en el directorio actual
    exportar_datos_a_csv(personas, ruta_csv)
    
    return personas  # Devolver la lista de personas por si se necesita más adelante

In [12]:
# Código para visualizar el CSV en el notebook después de generarlo
if __name__ == "__main__":
    # Número de INEs a generar
    num_ines = 300
    personas = generar_dataset(num_ines)
    
    # Mostrar el CSV generado
    import pandas as pd
    
    ruta_csv = "datos_ines.csv"
    if os.path.exists(ruta_csv):
        df = pd.read_csv(ruta_csv)
        print("\nVista previa del CSV generado:")
        display(df.head())

INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_001.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_002.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_003.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_004.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_005.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_006.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_007.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_008.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_009.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_010.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_011.jpg
INE falsa generada: C:\Users\Adrian\Desktop\data\INEs Dataset\ine_falsa_012.jpg
INE falsa generada: C:\Users\Adrian\Desk

,Archivo_Imagen,Nombre,Segundo Nombre,Apellido Paterno,Apellido Materno,Estado,Municipio,Colonia,Calle,Numero Ext.,Numero Int.,Codigo Postal,CURP,Clave Elector,Fecha de Nacimiento,Seccion,Vigencia Ano Inicio,Vigencia Ano Final,Ano Registro,Mes Registro
0,ine_falsa_001.jpg,MARIA,CRISTINA,CABALLERO,MERINO,JAL,VIEJA ISLAS MARSHALL,DE LA MONTANA NORTE,PROLONGACION COLOMBIA,S/N,NaN,22469,NZMC668219HANLOWU3,GYOJVG646605138YLP,16/02/1936,7751,2020,2030,2006,12
1,ine_falsa_002.jpg,JORGE,NaN,CORTES,LUCIO,MIC,VIEJA CÔTE D'IVOIRE,LOS BAJOS NORTE,CALLEJON VERACRUZ DE IGNACIO DE LA LLAVE MZ 5 LT,78,PISO 16,86080,FKVT018422HLZFRFT5,JYZRXM75539847S0YM,07/09/1935,6114,2021,2031,2013,4
2,ine_falsa_003.jpg,SERGIO,NaN,PELAYO,REGALADO,VER,VIEJA EGIPTO,LOS BAJOS SUR,PROLONGACION JALISCO,8672F,NaN,79474,PKWI178127HGYEFS39,MSVBLJ76123588T2HV,16/08/1989,7959,2020,2030,2023,5
3,ine_falsa_004.jpg,EVA,NaN,HEREDIA,CASAS,CDMX,SAN ERIC LOS BAJOS,LOS BAJOS NORTE,PERIFERICO CEPEDA KM,11.8,NaN,30229,KMHY152680MERVSR95,KAWXDT233011761AWP,21/01/2006,3562,2029,2039,2009,9
4,ine_falsa_005.jpg,CORNELIO,NaN,MORENO,ROJAS,TLAX,SAN AIDA DE LA MONTANA,LOS ALTOS NORTE,ANDADOR RODRIGEZ,8256D,PISO 4,27362,AYYC944500HRJHVSZ6,HJCETD400179338AZ3,18/11/2003,6598,2025,2035,2009,8
